# **Task B: Face Matching (Multi-Class Recognition with Distorted Inputs)**

```
Recommended to use Google Colab T4 GPU for faster execution
```

### 📌 **COMSYS 2025 | Task B – Face Matching under Distortions**
This task focuses on **robust face identity verification**.
Given a distorted face image, the model must determine whether it matches a known identity in the training gallery.

🧠 **Problem Statement:**
> Build a system that can verify if a distorted face image belongs to the same person as a reference set of clean images — even under challenging visual conditions such as blur, fog, or low light.

:**🎯 Task Objective:**
- Learn a face embedding space where:
  * Same person (original vs distorted) → **high similarity**
  * Different people → **low similarity**

- Use **cosine similarity** between embeddings to match distorted faces to identities.


| Component       | Choice                           |
| --------------- | -------------------------------- |
| Embedding Model | InsightFace (pretrained)         |
| Strategy        | Gallery-based Matching           |
| Distance Metric | Cosine Similarity                |
| Input           | `val/*/distortion/*.jpg`         |
| Output          | Label 1 (match) or 0 (non-match) |


In [ ]:
!pip install insightface onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 10.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.9 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1060436 sha256=3f075352d7c266b8196b4908991dbc78219b580dda5967e66bf3100cd4fb0b55
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface


### **Step 2: Load InsightFace Face Embedding Model**
In this task, we use **InsightFace** – a state-of-the-art face recognition library based on deep metric learning.
* The model (`buffalo_l`) outputs **512-dimensional embeddings** for each face.
* These embeddings are used to compare test and gallery images using **cosine similarity**.
* You can switch between **CPU** or **CUDA** as the compute backend.

In [ ]:
# -------------------------------
# 🧠 Step 1: Initialize InsightFace App for Embedding Extraction
# -------------------------------
from insightface.app import FaceAnalysis

# 🔧 Load InsightFace model with GPU (CUDA). Use 'CPUExecutionProvider' if GPU is unavailable.
app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider'])  # or ['CPUExecutionProvider']
app.prepare(ctx_id=0)  # ctx_id=0 refers to GPU:0

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:08<00:00, 33765.42KB/s]
/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [ ]:
### We can also use for less load, and fast check up
# app = FaceAnalysis(name="buffalo_sc", providers=['CPUExecutionProvider'])
# Compromise slightly with precision

### **📦 Step 3: Mount Google Drive and Extract Dataset**
In this step, we:
* Mount Google Drive to access dataset and saved models
* Unzip the provided dataset archive (`Comys_Hackathon5.zip`)
* Confirm that the main folder structure is extracted correctly

> ⚠️ This step is required before building the gallery or running any training/inference.



In [ ]:
# 📁 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')  # Grants access to files stored in Drive

# 🗜️ Define ZIP path (update path if needed)
zip_path = '/content/drive/MyDrive/COMSYS_25/Comys_Hackathon5.zip' ### Change the path as per your drive

# ✅ Unzip the dataset
!unzip -oq "$zip_path" -d /content/

# 📂 Confirm extraction
!ls /content/Comys_Hackathon5


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Task_A	Task_B


### 📂 **Step 4: Inspect Extracted Dataset Structure**
After unzipping the dataset, it's good practice to check that the folder hierarchy is correct.

In this step, we:

* Define the root folder (`/content/Comys_Hackathon5`)

* List the subfolders inside `Task_A/` and `Task_B/` to ensure data is correctly organized

In [ ]:
import os

# 📁 Define base path to extracted folder
base_path = '/content/Comys_Hackathon5'

# 🗂️ List contents of Task A and Task B
print("Task A folders:", os.listdir(os.path.join(base_path, 'Task_A')))
print("Task B folders:", os.listdir(os.path.join(base_path, 'Task_B')))

Task A folders: ['train', 'val']
Task B folders: ['train', 'val']


### 🧠 **Step 5: Define Embedding Functions with InsightFace**
In this step, we define two reusable functions to extract **512-dimensional face embeddings** from any input image.

These embeddings are later used to:
* Build a gallery of known faces (`train/`)
* Match distorted test faces (`val/`) against that gallery


In [ ]:
import cv2

# ✅ Simpler version using app.get() directly
def get_insight_embedding(image_path):
    """
    Extracts a 512-D face embedding using InsightFace high-level API.

    Args:
        image_path (str): Path to the image file.

    Returns:
        np.ndarray or None: Face embedding if face detected, else None.
    """
    img = cv2.imread(image_path)
    faces = app.get(img)
    if not faces:
        print("❌ No face detected:", image_path)
        return None
    return faces[0].embedding  # shape: (512,)


# 🔧 Lower-level, customizable version
def get_embedding_fast(img_path):
    """
    Extracts a face embedding using manual detection + alignment.
    Useful if you want more control over face preprocessing
    Args:
        img_path (str): Path to the image.
    Returns:
        np.ndarray or None: Face embedding vector.
    """
    img = cv2.imread(img_path)
    dets = app.det_model.detect(img)
    if len(dets[0]) == 0:
        print("⚠️ No face detected:", img_path)
        return None

    bboxes, landmarks = dets[0], dets[1]
    aligned = app.models['detection'].align(img, landmarks[0])
    emb = app.models['recognition'].get(aligned)
    return emb


### 🗂️ **Step 6: Build Face Embedding Gallery from Train and Val Folders**
In this step, we construct a gallery of known identities using clean, undistorted images from both:
* `Task_B/train/`
* `Task_B/val/` (non-distorted images only)

Each identity folder contains one or more face images.
We skip the `distortion/` folder and store multiple embeddings per identity.

This gallery will later be used to match against distorted test images.



In [ ]:
import os
from tqdm import tqdm
import numpy as np

# Path setup
train_dir = '/content/Comys_Hackathon5/Task_B/train'
val_dir   = '/content/Comys_Hackathon5/Task_B/val'

# We’ll combine clean images from both train and val
gallery_sources = [train_dir, val_dir]
gallery_embeddings = {}  # identity → [embedding1, embedding2, ...]

for source in gallery_sources:
    for identity in tqdm(os.listdir(source), desc=f"📁 Building gallery from {source.split('/')[-1]}"):
        identity_path = os.path.join(source, identity)
        if not os.path.isdir(identity_path):
            continue

        # Skip if identity already added
        if identity not in gallery_embeddings:
            gallery_embeddings[identity] = []

        # Load all clean images (skip distortion folder)
        for fname in os.listdir(identity_path):
            if fname == "distortion" or not fname.endswith(('.jpg', '.png')):
                continue

            img_path = os.path.join(identity_path, fname)
            emb = get_insight_embedding(img_path)  # or use get_embedding_fast
            if emb is not None:
                gallery_embeddings[identity].append(emb)

print(f"\n✅ Gallery built for {len(gallery_embeddings)} identities.")

📁 Building gallery from train:   1%|▏         | 11/877 [00:17<23:24,  1.62s/it]

❌ No face detected: /content/Comys_Hackathon5/Task_B/train/John_Bolton/John_Bolton_0016.jpg


📁 Building gallery from train:   9%|▉         | 82/877 [02:41<16:50,  1.27s/it]

❌ No face detected: /content/Comys_Hackathon5/Task_B/train/Tiger_Woods/Tiger_Woods_0023.jpg


📁 Building gallery from val:  53%|█████▎    | 132/250 [03:06<03:04,  1.56s/it]

❌ No face detected: /content/Comys_Hackathon5/Task_B/val/Padraig_Harrington/Padraig_Harrington_0004.jpg


📁 Building gallery from val: 100%|██████████| 250/250 [06:46<00:00,  1.63s/it]


✅ Gallery built for 1127 identities.


##💡 **Note:**
#### For Task B, since we use a pretrained embedding model (InsightFace) and do not perform model fine-tuning, the training set (train/) is only used to build the identity gallery.
#### Evaluation metrics (**Accuracy, Precision, Recall, F1**) are computed on the distorted validation images (val/) only.

### 🔁 **Step 7: Match Distorted Images to Gallery Using Cosine Similarity**
Now that we have:
* A gallery of reference face embeddings from `train/`
* A set of distorted images in `val/*/distortion/`

We compare each distorted test image to **all gallery identities** using cosine similarity.
The most similar identity is selected as the predicted label.

✅ **Matching Strategy:**
* If the most similar identity matches the ground-truth folder, and the similarity is above a threshold (`SIM_THRESHOLD`), we consider it a **correct match (label = 1)**.

* Else, it’s a **false match (label = 0)**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

SIM_THRESHOLD = 0.65  # 🧪 Tuneable threshold — 0.60 or 0.55 for lower strictness
val_distortion_dir = '/content/Comys_Hackathon5/Task_B/val'

processed = 0

y_true, y_pred = [], []

print("\n🔍 Matching distorted images to gallery...")

for identity in tqdm(os.listdir(val_distortion_dir), desc="🔁 Matching distorted samples"):
    dist_folder = os.path.join(val_distortion_dir, identity, 'distortion')

    # Skip if distortion folder doesn't exist or identity not in gallery
    if not os.path.isdir(dist_folder) or identity not in gallery_embeddings:
        continue

    for fname in os.listdir(dist_folder):
        if not fname.endswith(('.jpg', '.png')):
            continue

        test_path = os.path.join(dist_folder, fname)
        test_emb = get_insight_embedding(test_path)  # or get_embedding_fast
        if test_emb is None:
            continue  # Skip if face not detected

        best_score = -1
        predicted_identity = None

        # Compare test embedding to all gallery embeddings
        for gallery_id, emb_list in gallery_embeddings.items():
            sims = cosine_similarity([test_emb], emb_list)
            max_sim = sims.max()

            if max_sim > best_score:
                best_score = max_sim
                predicted_identity = gallery_id

        # ✅ Positive match: correct identity + high similarity
        is_match = int(predicted_identity == identity and best_score >= SIM_THRESHOLD)

        y_true.append(1)         # All distorted samples should match their folder
        y_pred.append(is_match)  # Match = 1, Wrong = 0

        print(f"[{identity}] → Predicted: {predicted_identity} | Score: {best_score:.4f} | Match: {is_match}")

        processed += 1

         # 💾 Auto-save after every 25 samples to prevent data loss
        if len(y_true) % 25 == 0:
            np.save('/content/drive/MyDrive/COMSYS_25/y_true_temp.npy', y_true)
            np.save('/content/drive/MyDrive/COMSYS_25/y_pred_temp.npy', y_pred)


🔍 Matching distorted images to gallery...


🔁 Matching distorted samples:   0%|          | 0/250 [00:00<?, ?it/s]

[Ghassan_Elashi] → Predicted: Ghassan_Elashi | Score: 0.7772 | Match: 1
[Ghassan_Elashi] → Predicted: Ghassan_Elashi | Score: 0.9683 | Match: 1
[Ghassan_Elashi] → Predicted: Ghassan_Elashi | Score: 0.9612 | Match: 1
[Ghassan_Elashi] → Predicted: Ghassan_Elashi | Score: 0.7996 | Match: 1
[Ghassan_Elashi] → Predicted: Pat_Burns | Score: 0.1973 | Match: 0
[Ghassan_Elashi] → Predicted: Ghassan_Elashi | Score: 0.7783 | Match: 1


🔁 Matching distorted samples:   0%|          | 1/250 [00:11<48:14, 11.62s/it]

[Ghassan_Elashi] → Predicted: Ghassan_Elashi | Score: 0.9760 | Match: 1
❌ No face detected: /content/Comys_Hackathon5/Task_B/val/Gloria_Gaynor/distortion/Gloria_Gaynor_0001_noisy.jpg
❌ No face detected: /content/Comys_Hackathon5/Task_B/val/Gloria_Gaynor/distortion/Gloria_Gaynor_0001_sunny.jpg
[Gloria_Gaynor] → Predicted: Gloria_Gaynor | Score: 0.6803 | Match: 1
[Gloria_Gaynor] → Predicted: Gloria_Gaynor | Score: 0.9667 | Match: 1
[Gloria_Gaynor] → Predicted: Gloria_Gaynor | Score: 0.9860 | Match: 1
[Gloria_Gaynor] → Predicted: Gloria_Gaynor | Score: 0.8906 | Match: 1


...

[Ben_Glisan] → Predicted: Ben_Glisan | Score: 0.9340 | Match: 1
[Martin_Luther_King_III] → Predicted: Martin_Luther_King_III | Score: 0.9656 | Match: 1
[Martin_Luther_King_III] → Predicted: Martin_Luther_King_III | Score: 0.9435 | Match: 1
❌ No face detected: /content/Comys_Hackathon5/Task_B/val/Martin_Luther_King_III/distortion/Martin_Luther_King_III_0001_sunny.jpg
[Martin_Luther_King_III] → Predicted: Martin_Luther_King_III | Score: 0.8571 | Match: 1
❌ No face detected: /content/Comys_Hackathon5/Task_B/val/Martin_Luther_King_III/distortion/Martin_Luther_King_III_0001_noisy.jpg
[Martin_Luther_King_III] → Predicted: Martin_Luther_King_III | Score: 0.7867 | Match: 1


🔁 Matching distorted samples: 100%|█████████▉| 249/250 [56:05<00:11, 11.82s/it]

[Martin_Luther_King_III] → Predicted: Martin_Luther_King_III | Score: 0.9898 | Match: 1
[Cate_Blanchett] → Predicted: Cate_Blanchett | Score: 0.9563 | Match: 1
[Cate_Blanchett] → Predicted: Cate_Blanchett | Score: 0.7906 | Match: 1
❌ No face detected: /content/Comys_Hackathon5/Task_B/val/Cate_Blanchett/distortion/Cate_Blanchett_0001_noisy.jpg
[Cate_Blanchett] → Predicted: Cate_Blanchett | Score: 0.9887 | Match: 1
❌ No face detected: /content/Comys_Hackathon5/Task_B/val/Cate_Blanchett/distortion/Cate_Blanchett_0003_sunny.jpg
❌ No face detected: /content/Comys_Hackathon5/Task_B/val/Cate_Blanchett/distortion/Cate_Blanchett_0003_rainy.jpg
❌ No face detected: /content/Comys_Hackathon5/Task_B/val/Cate_Blanchett/distortion/Cate_Blanchett_0001_sunny.jpg
❌ No face detected: /content/Comys_Hackathon5/Task_B/val/Cate_Blanchett/distortion/Cate_Blanchett_0004_rainy.jpg
❌ No face detected: /content/Comys_Hackathon5/Task_B/val/Cate_Blanchett/distortion/Cate_Blanchett_0003_noisy.jpg
❌ No face detected

🔁 Matching distorted samples: 100%|██████████| 250/250 [56:32<00:00, 13.57s/it]

[Cate_Blanchett] → Predicted: Cate_Blanchett | Score: 0.9610 | Match: 1


### **📊 Step 6: Evaluate Matching Performance (Accuracy, Precision, Recall, F1)**
### **Validation Results**
After comparing each distorted test image to the gallery, we compute standard evaluation metrics to assess performance:

| Metric       | Meaning in This Task                                                  |
| ------------ | --------------------------------------------------------------------- |
| ✅ Accuracy   | How many total matches were predicted correctly                       |
| 🎯 Precision | Of all predicted matches, how many were correct (low false positives) |
| 📈 Recall    | Of all actual matches, how many were correctly predicted              |
| 🏆 F1 Score  | Harmonic mean of Precision and Recall — best for imbalance            |

> ⚠️ We use `zero_division=0` to prevent runtime errors in case of 0 predictions or 0 ground truths.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("\n📊 Final Evaluation — Task B (Distorted Face Matching):")
print("✅ Accuracy  :", accuracy_score(y_true, y_pred))
print("🎯 Precision :", precision_score(y_true, y_pred, zero_division=0))
print("📈 Recall    :", recall_score(y_true, y_pred, zero_division=0))
print("🏆 F1 Score  :", f1_score(y_true, y_pred, zero_division=0))


📊 Final Evaluation — Task B (Distorted Face Matching):
✅ Accuracy  : 0.9146230699364214
🎯 Precision : 1.0
📈 Recall    : 0.9146230699364214
🏆 F1 Score  : 0.9554079696394687


### **🧪 Step 6: Face Matching Evaluation (Testing Shell for Task B)**
In this section, we evaluate the system's ability to match distorted face images to the correct identity.

This **testing shell** performs:
* ✅ Embedding extraction from clean `train/` images
* ✅ Matching of distorted `val/` images using cosine similarity
* ✅ Prediction of identity match (label = 1) or mismatch (label = 0)
* ✅ Computation of final evaluation metrics (Accuracy, Precision, Recall, F1)


##Testing (Optional)

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

# Set your two image paths from Google Drive
original_img_path  = '/content/drive/MyDrive/COMSYS/TaskB_testing/val/pic7/pic7.jpg'
distorted_img_path = '/content/drive/MyDrive/COMSYS/TaskB_testing/val/pic7/distortion/pic7_light.jpg'

# Load images for display (convert BGR to RGB)
img1 = cv2.imread(original_img_path)
img2 = cv2.imread(distorted_img_path)

# Check if images are loaded successfully
if img1 is None:
    print(f"❌ Error loading image: {original_img_path}")
elif img2 is None:
    print(f"❌ Error loading image: {distorted_img_path}")
else:
    # Get embeddings
    emb1 = get_insight_embedding(original_img_path)
    emb2 = get_insight_embedding(distorted_img_path)

    # Convert BGR to RGB for display
    img1_display = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2_display = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

    # Show both images side by side
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    axs[0].imshow(img1_display)
    axs[0].set_title("Original Image")
    axs[0].axis('off')
    axs[1].imshow(img2_display)
    axs[1].set_title("Distorted Image")
    axs[1].axis('off')
    plt.tight_layout()
    plt.show()

    # Now compare
    if emb1 is None or emb2 is None:
        print("❌ Face not detected in one or both images.")
    else:
        sim = cosine_similarity([emb1], [emb2])[0][0]
        print(f"🔗 Cosine Similarity: {sim:.4f}")

        threshold = 0.65
        if sim >= threshold:
            print("✅ Match Found (same person)")
        else:
            print("❌ No Match (different person)")

# END